# **Feed Forward Network**

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

**Menu**
- Device Config
- Hyper-parameters 
- Dataset & Dataloader
- Model Implementation
- Loss & optimizer
- Train & Test
- Save model

### **1. Device (CUDA)**

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### **2.Hyper-parameters**

In [4]:
'''Hyper-parameter'''
input_size = 28 * 28
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

### **3. Dataset & Dataloader**

In [8]:
train_dataset = torchvision.datasets.MNIST(root="./data/",
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root="./data/",
                                           train=False,
                                           transform=transforms.ToTensor(),
                                           download=True)

In [9]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

### **4. Model**

In [10]:
class FFN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FFN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [11]:
model = FFN(input_size, hidden_size, num_classes)

In [13]:
model.to(device)

FFN(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

### **5.Loss & Optimizer**

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### **6. Train**

In [18]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        '''move tensor to device'''
        images = images.reshape(-1, input_size).to(device)
        labels = labels.to(device)
        
        pred = model(images)
        loss = criterion(pred, labels)
        
        optimzier.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1755
Epoch [1/5], Step [200/600], Loss: 0.2771
Epoch [1/5], Step [300/600], Loss: 0.2180
Epoch [1/5], Step [400/600], Loss: 0.1410
Epoch [1/5], Step [500/600], Loss: 0.1801
Epoch [1/5], Step [600/600], Loss: 0.1019
Epoch [2/5], Step [100/600], Loss: 0.0744
Epoch [2/5], Step [200/600], Loss: 0.1015
Epoch [2/5], Step [300/600], Loss: 0.1151
Epoch [2/5], Step [400/600], Loss: 0.1056
Epoch [2/5], Step [500/600], Loss: 0.0893
Epoch [2/5], Step [600/600], Loss: 0.1773
Epoch [3/5], Step [100/600], Loss: 0.1561
Epoch [3/5], Step [200/600], Loss: 0.1340
Epoch [3/5], Step [300/600], Loss: 0.0405
Epoch [3/5], Step [400/600], Loss: 0.0334
Epoch [3/5], Step [500/600], Loss: 0.0610
Epoch [3/5], Step [600/600], Loss: 0.0228
Epoch [4/5], Step [100/600], Loss: 0.0486
Epoch [4/5], Step [200/600], Loss: 0.1080
Epoch [4/5], Step [300/600], Loss: 0.0123
Epoch [4/5], Step [400/600], Loss: 0.1114
Epoch [4/5], Step [500/600], Loss: 0.0740
Epoch [4/5], Step [600/600], Loss:

### **Test**

In [19]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.97 %


In [20]:
# Save the model checkpoint
torch.save(model.state_dict(), './model/FFN.ckpt')

In [26]:
model.load_state_dict(torch.load('./model/FFN.ckpt'))

<All keys matched successfully>

In [27]:
model

FFN(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)